In [12]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix
from keras import regularizers
import os
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.optimizers import Adam
import time
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import RMSprop
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
sv_df = pd.read_csv("/content/drive/MyDrive/female_vector_44100_216.csv")

In [35]:
from sklearn.utils import shuffle
sv_df_ = shuffle(sv_df)
sv_df_[:10]

,0,1,2,3,4,5,6,7,8,9,...,207,208,209,210,211,212,213,214,215,0.1
37767,-15.168839,-11.856977,-7.978371,-11.052394,-20.701372,-19.339035,-20.358574,-22.911783,-24.769085,-19.300816,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,female_surprise
15149,-24.253744,-5.422812,-2.132362,-8.046733,-19.422092,-22.753736,-25.379757,-27.125467,-29.760843,-31.886351,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,female_neutral
37439,-10.966912,-11.863079,-11.186807,-14.768787,-24.209686,-21.854713,-23.990040,-28.888256,-27.887529,-28.306042,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,female_nervous
4397,-3.155397,-7.594602,-25.923870,-19.899893,-18.538017,-20.327085,-19.050419,-21.113321,-24.633030,-24.748562,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,female_nervous
18452,-17.716707,-13.280382,-15.237415,-22.728642,-31.098152,-32.647793,-28.123049,-24.210602,-22.970926,-27.083109,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,female_angry
16577,-13.350271,-15.566784,-24.697838,-25.760176,-26.720448,-27.649488,-18.905672,-18.014961,-22.919176,-28.354124,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,female_surprise
21790,-12.904865,-14.977849,-21.625673,-29.710482,-33.785500,-36.828270,-32.973141,-31.845337,-31.518644,-32.347538,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,female_surprise
34416,-5.236537,-8.015001,-18.173910,-22.291679,-22.472683,-24.138615,-23.697775,-22.451660,-21.509539,-18.840839,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,female_nervous
3095,-21.227621,-20.914305,-17.532349,-6.997704,-6.426680,-12.872386,-15.933292,-17.175196,-8.136474,-6.980162,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,female_nervous
23965,-11.957691,-13.953912,-16.372334,-16.296951,-18.018969,-23.546749,-21.728107,-18.504494,-19.467775,-13.431597,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,female_nervous


In [36]:
sv_df_=sv_df_.fillna(0)

In [52]:
X = sv_df_.drop(sv_df_.iloc[:, [-1]], axis = 1)
y = sv_df_.iloc[:, [-1]]

In [53]:
X

,0,1,2,3,4,5,6,7,8,9,...,206,207,208,209,210,211,212,213,214,215
37767,-15.168839,-11.856977,-7.978371,-11.052394,-20.701372,-19.339035,-20.358574,-22.911783,-24.769085,-19.300816,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15149,-24.253744,-5.422812,-2.132362,-8.046733,-19.422092,-22.753736,-25.379757,-27.125467,-29.760843,-31.886351,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37439,-10.966912,-11.863079,-11.186807,-14.768787,-24.209686,-21.854713,-23.990040,-28.888256,-27.887529,-28.306042,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4397,-3.155397,-7.594602,-25.923870,-19.899893,-18.538017,-20.327085,-19.050419,-21.113321,-24.633030,-24.748562,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18452,-17.716707,-13.280382,-15.237415,-22.728642,-31.098152,-32.647793,-28.123049,-24.210602,-22.970926,-27.083109,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16025,-11.522718,-14.499363,-12.238410,-12.444851,-17.638987,-12.172926,-9.030269,-10.261932,-13.997803,-19.825565,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35617,-5.103899,-9.560563,-19.683208,-22.670454,-27.729748,-30.545712,-30.682207,-35.776768,-37.791878,-31.886288,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20050,-7.293814,-9.457795,-14.477392,-21.387764,-24.313494,-11.116791,-8.057163,-12.222748,-17.723774,-16.657394,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26365,-20.616203,-13.402781,-15.730389,-26.227600,-33.918259,-31.591637,-28.383533,-27.897543,-27.110119,-27.599773,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [55]:
X_train_, X_val, y_train_, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [56]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)
X_val = np.array(X_val)
y_val = np.array(y_val)

lb = LabelEncoder()
y_train = lb.fit_transform(y_train)
y_test = lb.transform(y_test)
y_val = lb.transform(y_val)

y_train =  to_categorical(y_train, num_classes=6)
y_test =  to_categorical(y_test, num_classes=6)
y_val =  to_categorical(y_val, num_classes=6)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [57]:
# 세번째 차원을 추가
# -> CNN은 3차원 배열을 입력을 받는다.
x_traincnn =np.expand_dims(X_train, axis=2)
x_testcnn= np.expand_dims(X_test, axis=2)
x_valcnn= np.expand_dims(X_val, axis=2)

In [59]:
scaler = StandardScaler()
X_train_normalized = scaler.fit_transform(X_train.reshape(-1, 1))
X_test_normalized = scaler.transform(X_test.reshape(-1, 1))
X_val_normalized = scaler.transform(X_val.reshape(-1, 1))

In [60]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('female_best_model_44100_216.h5', save_best_only=True)

In [61]:
y_train[0]

array([0., 1., 0., 0., 0., 0.], dtype=float32)

In [62]:
model = Sequential()

model.add(Conv1D(256, 5,padding='same',
                 input_shape=(216,1)))
model.add(Activation('relu'))
model.add(Dropout(0.3))

model.add(Conv1D(128, 5,padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.3))

model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
model.add(Dropout(0.3))

model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(6))
model.add(Activation('softmax'))

opt = RMSprop(learning_rate=0.00001)
model.summary()
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 216, 256)          1536      
                                                                 
 activation (Activation)     (None, 216, 256)          0         
                                                                 
 dropout (Dropout)           (None, 216, 256)          0         
                                                                 
 conv1d_1 (Conv1D)           (None, 216, 128)          163968    
                                                                 
 activation_1 (Activation)   (None, 216, 128)          0         
                                                                 
 dropout_1 (Dropout)         (None, 216, 128)          0         
                                                                 
 max_pooling1d (MaxPooling1  (None, 27, 128)           0

In [ ]:
cnnhistory=model.fit(x_traincnn, y_train, batch_size=64, epochs=100, validation_data=(x_testcnn, y_test))

In [ ]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Activation, Add, Multiply, GlobalAveragePooling1D, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import StandardScaler

# CSV 파일에서 데이터 로드 (가정: "audio_data.csv"는 음성 데이터 및 감정 레이블이 있는 파일)
audio_df = pd.read_csv("/content/drive/MyDrive/female_vector_44100_216.csv")

# 데이터 섞기
audio_df = shuffle(audio_df)

# 결측값 0으로 채우기 (만약 결측값이 존재한다면)
audio_df = audio_df.fillna(0)

# Features (X)와 Labels (y)로 데이터 나누기
X = audio_df.drop(audio_df.columns[-1], axis=1)
y = audio_df.iloc[:, -1]

# Label 인코딩
lb = LabelEncoder()
y = lb.fit_transform(y)
y = to_categorical(y, num_classes=6)  # num_classes는 감정 클래스의 수에 맞게 조정

# 데이터를 학습, 검증, 테스트 셋으로 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Conv1D 입력을 위한 차원 확장
x_train_cnn = np.expand_dims(X_train.values, axis=2)
x_test_cnn = np.expand_dims(X_test.values, axis=2)
x_val_cnn = np.expand_dims(X_val.values, axis=2)

# StandardScaler를 사용하여 훈련 데이터 정규화
scaler = StandardScaler()
X_train_normalized = scaler.fit_transform(x_train_cnn.reshape(-1, 1))
X_test_normalized = scaler.transform(x_test_cnn.reshape(-1, 1))


scaler = StandardScaler()
X_train_normalized = scaler.fit_transform(x_train_cnn.reshape(-1, 1))
X_test_normalized = scaler.transform(x_test_cnn.reshape(-1, 1))
X_val_normalized = scaler.transform(x_val_cnn.reshape(-1, 1))

# WaveNet 모델 정의
model = Sequential()

for i in range(3):  # 조절 가능: residual block 개수
    model.add(Conv1D(256, 2, dilation_rate=2**i, padding='causal', input_shape=(X_train.shape[1], 1)))
    model.add(Activation('relu'))
    model.add(Conv1D(128, 2, dilation_rate=2**i, padding='causal'))
    model.add(Activation('relu'))

    # WaveNet의 residual block 구조 (간략화)
    residual = model.layers[-1].output
    x = Conv1D(128, 2, dilation_rate=2**i, padding='causal')(residual)
    x = Activation('relu')(x)
    x = Conv1D(128, 2, dilation_rate=2**i, padding='causal')(x)
    x = Activation('relu')(x)
    x = Multiply()([x, residual])
    x = Add()([x, residual])

model.add(GlobalAveragePooling1D())
model.add(Dense(6, activation='softmax'))  # num_classes는 감정 클래스의 수에 맞게 조정

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 콜백 정의
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_audio_model.h5', save_best_only=True)

# 모델 학습
history = model.fit(x_train_cnn, y_train, epochs=50, batch_size=64, validation_data=(x_val_cnn, y_val), callbacks=[early_stopping, model_checkpoint])

# 테스트 셋에서 모델 평가
test_loss, test_accuracy = model.evaluate(x_test_cnn, y_test)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')


Epoch 1/50
460/460 [==============================] - 230s 497ms/step - loss: 1.6600 - accuracy: 0.2760 - val_loss: 1.5871 - val_accuracy: 0.3136
Epoch 2/50


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


460/460 [==============================] - 228s 495ms/step - loss: 1.5790 - accuracy: 0.3345 - val_loss: 1.5467 - val_accuracy: 0.3561
Epoch 3/50
460/460 [==============================] - 224s 487ms/step - loss: 1.5344 - accuracy: 0.3702 - val_loss: 1.4834 - val_accuracy: 0.4018
Epoch 4/50
460/460 [==============================] - 220s 478ms/step - loss: 1.5014 - accuracy: 0.3879 - val_loss: 1.4783 - val_accuracy: 0.3947
Epoch 5/50
460/460 [==============================] - 220s 478ms/step - loss: 1.4680 - accuracy: 0.4050 - val_loss: 1.4751 - val_accuracy: 0.3944
Epoch 6/50
460/460 [==============================] - 224s 486ms/step - loss: 1.4465 - accuracy: 0.4148 - val_loss: 1.4493 - val_accuracy: 0.4168
Epoch 7/50
460/460 [==============================] - 221s 479ms/step - loss: 1.4219 - accuracy: 0.4283 - val_loss: 1.4283 - val_accuracy: 0.4230
Epoch 8/50
460/460 [==============================] - 222s 483ms/step - loss: 1.4094 - accuracy: 0.4382 - val_loss: 1.4885 - val_accura

In [ ]:
# 간단한 RNN 모델 정의 (Bidirectional LSTM 사용, 4층, 12개의 출력 클래스)
model = Sequential()
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.3))
model.add(Dense(6, activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(x_traincnn, y_train,
          epochs=100,
          batch_size=64,
          validation_data=(x_valcnn, y_val),
          callbacks=model_checkpoint)

In [57]:
y_train_dim1 = y_train.ravel()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Label Encoding 및 1차원으로 변환
lb = LabelEncoder()
y_train_dim1 = lb.fit_transform(y_train).ravel()
y_test_dim1 = lb.transform(y_test).ravel()

# SVM 모델 생성
svm_model = SVC()

# Grid Search를 위한 파라미터 그리드 설정
param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [0.001, 0.01, 0.1, 1]}

# Grid Search 수행
grid_search = GridSearchCV(svm_model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train_dim1)

# 최적의 하이퍼파라미터 출력
print("Best Parameters:", grid_search.best_params_)

# 최적 모델 훈련 및 예측
best_svm_model = grid_search.best_estimator_
y_pred = best_svm_model.predict(X_test)

# 정확도 출력
accuracy = accuracy_score(y_test_dim1, y_pred)
print("Accuracy:", accuracy)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Activation, Add, Multiply, GlobalAveragePooling1D, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# CSV 파일에서 데이터 로드
sv_df = pd.read_csv("/content/drive/MyDrive/음성/female_vector_44100.csv")

# 데이터 섞기
sv_df_ = shuffle(sv_df)

# 결측값 0으로 채우기
sv_df_ = sv_df_.fillna(0)

# Features (X)와 Labels (y)로 데이터 나누기
X = sv_df_.drop(sv_df_.columns[-1], axis=1)
y = sv_df_.iloc[:, -1]

# Label 인코딩
lb = LabelEncoder()
y = lb.fit_transform(y)
y = to_categorical(y, num_classes=6)  # num_classes는 감정 클래스의 수에 맞게 조정

# 데이터를 학습, 검증, 테스트 셋으로 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Conv1D 입력을 위한 차원 확장
x_train_cnn = np.expand_dims(X_train.values, axis=2)
x_test_cnn = np.expand_dims(X_test.values, axis=2)
x_val_cnn = np.expand_dims(X_val.values, axis=2)

# WaveNet 모델 정의
model = Sequential()

for i in range(3):  # 조절 가능: residual block 개수
    model.add(Conv1D(64, 2, dilation_rate=2**i, padding='causal', input_shape=(X_train.shape[1], 1)))
    model.add(Activation('relu'))
    model.add(Conv1D(64, 2, dilation_rate=2**i, padding='causal'))
    model.add(Activation('relu'))

model.add(GlobalAveragePooling1D())
model.add(Dense(6, activation='softmax'))  # 6은 감정 클래스의 수에 맞게 조정

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 콜백 정의
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('female_best_model_44100.h5', save_best_only=True)

# 모델 학습
history = model.fit(x_train_cnn, y_train, epochs=50, batch_size=64, validation_data=(x_val_cnn, y_val), callbacks=[early_stopping, model_checkpoint])

# 테스트 셋에서 모델 평가
test_loss, test_accuracy = model.evaluate(x_test_cnn, y_test)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')